In [ ]:
import pandas as pd
import numpy as np
import warnings
import locale
import pyodbc
import tkinter as tk
from tkinter import messagebox
from datetime import datetime

warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

hoy = datetime.now()
año = hoy.year
mes = hoy.month

mes_nombre = hoy.strftime('%b').upper()[:3] # ENE
fecha = f'{mes_nombre}{str(año)[2:]}' # ENE24
fecha = f'{año}{str(mes).zfill(2)}' # 202401


asignacion_path = f'bases/asignacion/{fecha}/origen/base_asignacion_{fecha}.txt'

asignacion_pagos_path = f'bases/asignacion/{fecha}/base_asignacion_{fecha}.xlsx'
asignacion_pagos_path_txt = f'bases/asignacion/{fecha}/base_asignacion_{fecha}.txt'

asignacion_camp_path = f'bases/asignacion/{fecha}/base_asignacion_{fecha}_CAMP.xlsx'
asignacion_camp_path_txt = f'bases/asignacion/{fecha}/base_asignacion_{fecha}_CAMP.txt'

asignacion_gest_path_txt = f'bases/asignacion/{fecha}/base_asignacion_{fecha}_GEST.txt'

stock_judicial_path = f'bases/asignacion/{fecha}/stocks/StockJudicial.csv'
stock_castigo_path = f'bases/asignacion/{fecha}/stocks/StockCastigos.csv'

print(asignacion_path)
print(asignacion_pagos_path)
print(asignacion_camp_path)
print(asignacion_gest_path_txt)
print(stock_judicial_path)
print(stock_castigo_path)

In [ ]:
def get_connection(server_name: str, database_name: str, username: str = None, password: str = None) -> pyodbc.Connection:
    try:
        if username and password:
            conn = pyodbc.connect(
                'DRIVER={ODBC Driver 17 for SQL Server};'
                f'SERVER={server_name};'
                f'DATABASE={database_name};'
                f'UID={username};'
                f'PWD={password};'
            )
            i = 0
        else:
            conn = pyodbc.connect(
                'DRIVER={ODBC Driver 17 for SQL Server};'
                f'SERVER={server_name};'
                f'DATABASE={database_name};'
                'Trusted_Connection=yes;'
            )
            i = 1
    except Exception as e:
        print('Error de conexión:', str(e))
        return
    finally:
        if conn:
            if i == 1:
                print('Conexión exitosa: Autenticación de Windows')
            else:
                print('Conexión exitosa: UserCobranza')
            return conn

def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

def update_columns(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    df.columns = clean_columns(df.columns)
    
    if df['CAPITAL'].dtype == 'object':
        df['CAPITAL'] = df['CAPITAL'].astype(str).str.replace(',', '').astype(float)
    df['CAPITAL'] = pd.to_numeric(df['CAPITAL'], errors='coerce')
    df['CAPITAL'].fillna(0, inplace=True)
    
    if df['CAMP'].dtype == 'object':
        df['CAMP'] = df['CAMP'].str.extract(r'(\d+)%').astype(float) / 100
    df['CAMP'] = pd.to_numeric(df['CAMP'], errors='coerce')
    df['CAMP'].fillna(0, inplace=True)
    
    df['TIPO_FONDO'].fillna('NULL', inplace=True)
    df['MARCA'].fillna('NULL', inplace=True)
    df['TRAMO'].fillna('NULL', inplace=True)
    df['SEGMENTO'].fillna('NULL', inplace=True)
    
    conteo_cc = df['COD_CENTRAL'].value_counts()
    df['FLAG'] = df['COD_CENTRAL'].map(conteo_cc)
    
    df['TIPO_FONDO'] = df['TIPO_FONDO'].str.strip()
    
    if 'REACTIVA_KST' in df['TIPO_FONDO'].unique():
        df['TIPO_FONDO'] = df['TIPO_FONDO'].replace({'REACTIVA_KST': 'REACTIVA'})
    
    df.sort_values(by=['AGENCIA', 'MARCA', 'TRAMO', 'SEGMENTO', 'COD_CENTRAL', 'CONTRATO'], ascending=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df[cols]

def cartera_extrajudicial(df: pd.DataFrame) -> pd.DataFrame:
        df['TIPO_CARTERA'] = np.where(
            (df['MARCA'] == 'EXJ'), 'UNSECURED', df['TIPO_CARTERA']
        )
        print('Asignacion EXJ Actualizada')
        print('--------------------------------------------')
        print(df[['MARCA', 'TIPO_CARTERA']].value_counts(dropna=False).sort_index())
        print('\n')
        
        return df

def cartera_judicial(df: pd.DataFrame) -> pd.DataFrame:
    df_cartera_judicial = pd.read_csv(stock_judicial_path, sep=';', encoding='utf-8', dtype={'CONTRATO': str})
    cols_judicial = ['CONTRATO', 'TIPO_CARTERA']
    df_cartera_judicial = df_cartera_judicial[cols_judicial]
    df_cartera_judicial['CONTRATO'] = df_cartera_judicial['CONTRATO'].apply(lambda x: x[-18:] if pd.notna(x) else x)
    
    print('Stock Judicial')
    print('--------------------------------------------')
    print(df_cartera_judicial['TIPO_CARTERA'].value_counts(dropna=False).sort_index())
    print('\n')
    
    df_bc = df[df['MARCA'] == 'BC']
    df_bc = df_bc.merge(df_cartera_judicial, on='CONTRATO', how='left', suffixes=('', '_judicial'))
    
    tipo_cartera_mapping = df_bc.set_index('CONTRATO')['TIPO_CARTERA_judicial'].to_dict()
    df['TIPO_CARTERA'] = np.where(
        df['MARCA'] == 'BC',
        df['CONTRATO'].map(tipo_cartera_mapping),
        df['TIPO_CARTERA']
    )
    print('Asignacion BC Actualizada')
    print('--------------------------------------------')
    print(df[['MARCA', 'TIPO_CARTERA']].value_counts(dropna=False).sort_index())
    print('\n')
    
    return df

def cartera_castigo(df: pd.DataFrame) -> pd.DataFrame:
    df_cartera_castigo = pd.read_csv(stock_castigo_path, sep=';', encoding='utf-8', dtype={'CONTRATO': str})
    df_cartera_castigo.drop(columns=['TIPO_CARTERA'], inplace=True)
    df_cartera_castigo.rename(columns={'GRUPO': 'TIPO_CARTERA'}, inplace=True)
    cols_castigo = ['CONTRATO', 'TIPO_CARTERA']
    df_cartera_castigo = df_cartera_castigo[cols_castigo]
    df_cartera_castigo['CONTRATO'] = df_cartera_castigo['CONTRATO'].apply(lambda x: x[-18:] if pd.notna(x) else x)
    
    print('Stock Castigos')
    print('--------------------------------------------')
    print(df_cartera_castigo['TIPO_CARTERA'].value_counts(dropna=False).sort_index())
    print('\n')
    
    df_kstbc = df[df['MARCA'] == 'KSTBC']
    df_kstbc = df_kstbc.merge(df_cartera_castigo, on='CONTRATO', how='left', suffixes=('', '_castigo'))
    
    tipo_cartera_mapping = df_kstbc.set_index('CONTRATO')['TIPO_CARTERA_castigo'].to_dict()
    df['TIPO_CARTERA'] = np.where(
        df['MARCA'] == 'KSTBC',
        df['CONTRATO'].map(tipo_cartera_mapping),
        df['TIPO_CARTERA']
    )
    print('Asignacion KSTBC Actualizada')
    print('--------------------------------------------')
    print(df[['MARCA', 'TIPO_CARTERA']].value_counts(dropna=False).sort_index())
    print('\n')
    
    return df

def cartera_cdt(df: pd.DataFrame) -> pd.DataFrame:
    df['TIPO_CARTERA'] = np.where(
        (df['MARCA'] == 'CDT') & (df['AGENCIA'] == 'MORNESE MORA'), 'UNSECURED', df['TIPO_CARTERA']
    )
    print('Asignacion CDT Actualizada')
    print('--------------------------------------------')
    print(df[['MARCA', 'TIPO_CARTERA']].value_counts(dropna=False).sort_index())
    print('\n')
    
    return df

In [ ]:
conexion = get_connection(server_name='118.180.55.67', database_name='CentroCobranzas', username='usrcobranza', password='P@ssw0rd')
query = (
    f"""
        SELECT
            agencia AS AGENCIA, 
            codigo AS COD_CENTRAL, 
            cont_18 AS CONTRATO, 
            clave AS MARCA, 
            FOCO AS TRAMO, 
            CLUSTER AS SEGMENTO, 
            nombre AS NOMBRE_CLIENTE, 
            moneda AS MONEDA, 
            CAPITAL, 
            Camp AS CAMP, 
            TIPO_FONDO, 
            TIPO_CARTERA, 
            PRODUCTO, 
            fechadatos as FECHA_DATOS
        FROM CENTROCOBRANZAS.DBO.PD_DISTRIBUCION_TARDIA_HOY
    """
)

df_asignacion = pd.read_sql(query, conexion)
df_asignacion.to_csv(asignacion_path, sep='|', encoding='utf-8', index=False)
print(f'Asignación {fecha}:', df_asignacion.shape)

conexion.close()

In [ ]:
# Leer base asignacion #

root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar base asignacion?', icon='warning')

if result == 'yes':
    df_asignacion = pd.read_csv(asignacion_path, sep='|', encoding='utf-8', dtype={'COD_CENTRAL': str, 'CONTRATO': str})
    df_asignacion.columns = clean_columns(df_asignacion.columns)
    print(df_asignacion.columns)
    print('Base Asignación:', df_asignacion.shape)

root.destroy()

In [ ]:
# Separar base asignacion por agencia #

root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', 'Separar base asignación por agencia?', icon='warning')

if result == 'yes':
    def separar_asignacion(df: pd.DataFrame, agencia: str) -> pd.DataFrame:
        df_asignacion_agencia = df[df['AGENCIA'] == agencia]
        print(f'Base de Asignacion: {agencia} - {df_asignacion_agencia.shape}')
        print('--------------------------------------------')
        print(df_asignacion_agencia[['AGENCIA', 'CLAVE']].value_counts(dropna=False))
        print('\n')
        
        return df_asignacion_agencia
    
    df_asignacion_rj = separar_asignacion(df_asignacion, 'ASESCOM RJ')
    df_asignacion_clasa = separar_asignacion(df_asignacion, 'CLASA MORA')
    df_asignacion_mornese = separar_asignacion(df_asignacion, 'MORNESE MORA')

root.destroy()

In [ ]:
df_asignacion_test = df_asignacion.copy()

cols_required = ['AGENCIA', 'COD_CENTRAL', 'CONTRATO', 'MARCA', 'TRAMO', 'SEGMENTO', 'MONEDA', 'CAPITAL', 'CAMP', 'TIPO_FONDO', 'TIPO_CARTERA', 'FLAG', 'NOMBRE_CLIENTE', 'PRODUCTO']
df_asignacion_test = update_columns(df_asignacion_test, cols_required)
df_asignacion_test.shape

In [ ]:
# Cargar stocks #

root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', 'Cargar stocks con base asignación?', icon='warning')

if result == 'yes':
    print('---------------------- INICIO ----------------------')
    print(df_asignacion_test[['MARCA', 'TIPO_CARTERA']].value_counts(dropna=False).sort_index())
    print('\n')
    
    df_asignacion_test = cartera_extrajudicial(df_asignacion_test)
    df_asignacion_test = cartera_judicial(df_asignacion_test)
    df_asignacion_test = cartera_castigo(df_asignacion_test)
    df_asignacion_test = cartera_cdt(df_asignacion_test)
    
    print('---------------------- FIN ----------------------')
    print(df_asignacion_test[['MARCA', 'TIPO_CARTERA']].value_counts(dropna=False).sort_index())
    print('\n')

root.destroy()

In [ ]:
cols_asig_pagos = ['AGENCIA', 'COD_CENTRAL', 'CONTRATO', 'MARCA', 'TRAMO', 'SEGMENTO', 'MONEDA', 'CAPITAL', 'CAMP', 'TIPO_FONDO', 'TIPO_CARTERA', 'FLAG', 'NOMBRE_CLIENTE', 'PRODUCTO']
df_asignacion_pagos = df_asignacion_test[cols_asig_pagos]

cols_asig_camp = ['AGENCIA', 'COD_CENTRAL', 'CONTRATO', 'MARCA', 'MONEDA', 'CAPITAL', 'CAMP', 'TIPO_FONDO', 'TIPO_CARTERA', 'NOMBRE_CLIENTE']
df_asignacion_camp = df_asignacion_test[cols_asig_camp]

cols_asig_gest = ['AGENCIA', 'COD_CENTRAL', 'CONTRATO', 'MARCA', 'TRAMO', 'SEGMENTO', 'CAPITAL', 'TIPO_CARTERA']

In [ ]:
#df_asignacion_pagos.to_excel(asignacion_pagos_path, sheet_name='BASE', index=False)
df_asignacion_pagos.rename(columns={'COD_CENTRAL': 'CC', 'MARCA': 'CARTERA'}, inplace=True)
df_asignacion_pagos.to_csv(asignacion_pagos_path_txt, sep='|', encoding='utf-8', index=False)
print('Base Asignación:', df_asignacion_pagos.shape)

In [ ]:
# Generar base de asignación para campañas #

root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar base asignacion para campañas?', icon='warning')

if result == 'yes':
    df_asignacion_camp.to_excel(asignacion_camp_path, index=False, sheet_name='BASE')
    df_asignacion_camp.to_csv(asignacion_camp_path_txt, index=False, sep='|', encoding='utf-8')
    print('Base Asignación Campañas:', df_asignacion_camp.shape)

root.destroy()

In [ ]:
# Generar base de asignación para gestiones diarias #

root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar base asignacion para gestiones?', icon='warning')

if result == 'yes':
    df_asignacion_gest = df_asignacion_test[cols_asig_gest]
    df_asignacion_gest = df_asignacion_gest[df_asignacion_gest['TIPO_CARTERA'] == 'UNSECURED']
    df_asignacion_gest = df_asignacion_gest[df_asignacion_gest['MARCA'].isin(['BC', 'EXJ', 'KSTBC'])]
    df_asignacion_gest = df_asignacion_gest[df_asignacion_gest['TRAMO'].isin(['C1', 'C2', 'C3'])]
    df_asignacion_gest = df_asignacion_gest[df_asignacion_gest['AGENCIA'].isin(['ASESCOM RJ', 'CLASA MORA', 'MORNESE MORA'])]
    df_asignacion_gest.drop(columns=['TIPO_CARTERA'], inplace=True)
    
    df_asignacion_gest.to_csv(asignacion_gest_path_txt, sep='|', encoding='utf-8', index=False)
    print('Base Asignación Gestiones:', df_asignacion_gest.shape)

root.destroy()

In [ ]:
df_asignacion_gest['SEGMENTO'].value_counts(dropna=False)